# 1. Problem 1: Evolution of Sales Volume

### <ins>General notes for the exam: </ins>

You can access all the data via your usual import package (e.g PANDAS in python):

In Python:
```
import pandas as pd
df_product = pd.read_csv('product_data.csv')
```

In R:
```
df_product <- read.csv('product_data.csv')
```

All needed packages are available in the test:
Non exhaustive list: pandas, numpy, sklearn, scipy, ...

Do not hesitate to <ins>***COMMENT***<ins> on your code and explain your ideas.

Try and answer all questions fully. If running out of time, please note that questions 6, 8 and 9 deliver the most points in the scoring system.

Throughout this entire exam, your goal will be to help a grocery company to better use its marketing campaigns. 

### <ins>How to debug your code: </ins>

To see the result of any print statements, you should:
1. Choose the tab "Custom input" next to the "Test Results" tab.
2. Fill in the text box that appears with: 'BCG'. You do not have to put your code in this box.


### <ins>Description of the data provided on Section 1: </ins>

You are provided two datasets containing data about:
1. customer_data: containing data about customers
2. orders_data: containing data about orders

You can access them with the following snippet:
```
customers_df = pd.read_csv('customer_data.csv')
orders_df = pd.read_csv('order_data.csv')
```

The table `customer_data` containing the following columns:
- `customer_id`: id of the customer
- `birth_date`: birth date of the customer 
- `acquisition_channel`: marketing channel through which the customer was acquired.

The table `orders_data` containing the following columns:
- `order_id`: id of the order
- `transacion_date`: date of the transaction
- `product`: ordered product
- `price`: price of the bought product in $
- `quantity`: quantity ordered of the product
- `customer_id`: customer issuing the order

Before getting modeling, we would like to do some preliminary analysis to understand better the data we have.


In [2]:
import pandas as pd
customers_df = pd.read_csv('customer_data.csv')
orders_df = pd.read_csv('order_data.csv')

In [3]:
customers_df.head()

,customer_id,birth_date,acquisition_channel
0,4c7883248ada4d8d90ef20d0e9af2835,2001/3/12-6:55:53,Web
1,b27bfda31f454a0da6172998f58cdabf,1993/8/5-13:34:11,Radio
2,5d58141206194dc5bc14c4e8afba6603,1969/6/8-17:31:10,Billboard
3,639d8161f78848e083adc45b9b39d455,1948/6/22-0:49:29,Billboard
4,5a2909e4342140a4a3f979bc454fa3e0,1993/2/9-17:8:36,Radio


In [4]:
orders_df.head()

,transacion_date,product,price,quantity,orders_id,customer_id
0,12/1/2010 8:26,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,604044f8f75c4cfea4d728a45f8ac734,4c7883248ada4d8d90ef20d0e9af2835
1,12/1/2010 8:26,WHITE METAL LANTERN,3.39,6,7d6e32355659484aa66b64f6bdd708d7,b27bfda31f454a0da6172998f58cdabf
2,12/1/2010 8:26,CREAM CUPID HEARTS COAT HANGER,2.75,8,27339e14b50c43e9984c434d1939f983,b27bfda31f454a0da6172998f58cdabf
3,12/1/2010 8:26,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,8ac28c0797be4d1d826086f606ad7add,b27bfda31f454a0da6172998f58cdabf
4,12/1/2010 8:26,RED WOOLLY HOTTIE WHITE HEART.,3.39,6,b87ca12652d14c66bd86e6b80f28b279,b27bfda31f454a0da6172998f58cdabf


### <ins>Problem 1:</ins>

What is the relative difference between the total sales on 2018 and 2017 ?
The relative difference is defined as: (sales_2018/sales_2017 - 1)
Sales are calculated as an ammount in $

In [69]:
big_table = pd.merge(orders_df, customers_df, how="left", on="customer_id")
big_table['amount'] = big_table['price']*big_table['quantity']
big_table['transacion_date'] = pd.to_datetime(big_table['transacion_date'], format='%d/%m/%Y %H:%M')
big_table['year'] = big_table['transacion_date'].dt.year
#big_table.head()
big_table.groupby('year')['amount'].sum()

year
2010    167910.51
Name: amount, dtype: float64

# 2. Problem 2: Population type per acquisition channel

### <ins>Problem 2:</ins>

What is the median age per acquisition channel?
Please return a pandas dataframe containing the following columns:
- acquisition_channel: contains the channel name (radio, tv, ...)
- median_age: median age of the given channel

N.B: The median should be calculated on an integer 'age': convert the age to an integer before calculating the median.


In [100]:
big_table = pd.merge(orders_df, customers_df, how="left", on="customer_id")
big_table['birth_date'] = pd.to_datetime(big_table['birth_date'], format='%Y/%m/%d-%H:%M:%S', errors='coerce')
big_table.dropna(inplace=True)
big_table['birth_year'] = big_table['birth_date'].dt.year
big_table['age'] = 2022 - big_table["birth_year"]
big_table["age"].head()

big_table.groupby('acquisition_channel')['age'].agg("mean")

acquisition_channel
Billboard    51.350659
Radio        53.328402
TV           48.223717
Web          45.144148
Name: age, dtype: float64

In [101]:
res = big_table.where(big_table.quantity <= 0)[['product', 'quantity']]
res.dropna(inplace=True)
res.head()

,product,quantity
141,Discount,-1.0
154,SET OF 3 COLOURED FLYING DUCKS,-1.0
235,PLASTERS IN TIN CIRCUS PARADE,-12.0
236,PACK OF 12 PINK PAISLEY TISSUES,-24.0
237,PACK OF 12 BLUE PAISLEY TISSUES,-24.0


# 3. Problem 3: Popular product within millennials

### <ins>Problem 3:</ins>

What is the most popular product (in terms of number of sold units) among the millennials (born between 1981 and 1996 incl.) ?

In [103]:
big_table.dropna(inplace=True)
t = big_table.where((big_table.birth_year>=1981) & (big_table.birth_year<=1996))[["product", "quantity"]]
t.dropna(inplace=True)
# t.head(20)
t.groupby('product').agg("sum").sort_values(by=['quantity'], ascending=False).head(5)

,quantity
product,
RED HARMONICA IN BOX,1484.0
PACK OF 12 LONDON TISSUES,677.0
WHITE HANGING HEART T-LIGHT HOLDER,345.0
LADS ONLY TISSUE BOX,340.0
JUMBO BAG RED RETROSPOT,301.0


# 4. Linear regression (1/2)

## Section 2: Linear Regression 

We would like to understand which marketing channel is the most effective.
For that, the marketing department of our client provided us with a dataset containing weekly spends on each channel and the revenue generated that week during the 3 last years.

The data in on a tabular format with the following columns:
- week: a week identifier
- spends_tv: spendings on tv marketing campaign that week
- spends_radio: spendings on ads on the radio
- spends_web: spendings on web ads
- spends_billboard: spendings on phisical ads on billboards
- revenue: revenue generated during this week

Your colleague had the idea of treating this problem as regression problem where he tries to estimate the revenue as linear function of spendings.
He has performed a linear regression and obtained the following results: 



OLS Results
================================================================

| Variable    | Value       |
| ----------- | ----------- |
| Dep. Variable      | revenue       |
| Model   | OLS        |
| Method   | Least Squares        |
| Date   | Mon, 14 May 2018        |
| Time   | 21:48:12        |
| No. Observations   | 156        |
| Df Model   | 3        |
| Covariance type   | nonrobust        |
| R-squared   | 0.816        |
| Adj. R-squared   | 0.712        |
| F-statistic   | 6.646        |
| Prob(F-statistic)   | 0.00157        |
| Log-Likelihood   | -12.974        |


================================================================


| |coef| str err | t | P>\|t\| |
|--- | ---| --- | --- | --- |
|spends_tv | 10454.7| 197.2 | 53.02 | <0.00001 |
|spends_radio | 5984.2| 959.3 | 6.238 | 0.0041543 |
|spends_web | 8324.1| 134.5 | 61.89_ | <0.00001 |
|spends_billboard | 6278.5| 434.1 | 14.46 | <0.000359 |
|const | 30332.2| 202.1 | 150.1 | <0.00001 |

================================================================


| Variable    | Value       |
| ----------- | ----------- |
| Omnibus      | 0.176       |
| Prob(Omnibus)   | 0.916        |
| Skew   | 0.141        |
| Kurtosis   | 2.786        |
| Durbin-Watson   | 2.346        |
| Jarque-Bera (JB)   | 0.167        |
| Prob(JB)   | 0.920        |
| Cond. No.   | 176.        |


================================================================

Warnings:
[1] Standard errors assume that the convariance matrix of the errors is correctly specified.


### <ins>Problem 4:</ins>
<ins>Question 1:</ins>

With certainty, can you provide the MOST effective marketing channel?


# ASK HERE

### <ins>Problem 5:</ins>
<ins>Question 2:</ins>

With certainty, can you provide the LEAST effective marketing channel?
Pick **ONE** option
- TV
- Radio 
- Web
- Billboard
- Can't say 


# ASK HERE

# 6. Regression model to predict revenues

## Section 2: Build a regression model

The marketing department with which we are working want to send personalised promotions to targeted customers. They need help from us to get the highest value customers: customers who will generate the most revenues.

For this, you are asked to create a **regression model** that predicts the demand for a given customer and year. The **target value** is **revenue** which is equal to the **price \* quantity** <ins>**agregated at year level**<ins>

Important Note in this section:

Do not hesitate to write comments and to modularize your code.
You will be evaluated both on the result and the quality of your code.
If you get stuck in a question, do not hesitate to move on the next question.
Points are assigned independently for each question.



**Question 1:**

Using the two tables from Question 1 (orders_data and customer_data), create an aggregated table of revenus by year and customer_id.



In [ ]:
# Code

**Question 2:**
Create the following features on the aggregated table from question 1:
- age: customer age
- prev_year_revenue: revenue generated by the customer on the previous year
- prev_year_nb_products: number of distinct products bought by the customer on the previous year


In [ ]:
# Code

**Question 3:**
Can you add some other features that may help the model?
We expect you to add at least 2 features.


In [ ]:
# Code

**Question 4:**
We will split the data on a training and test sets. The test set should correspond to orders in the year 2018, and the other years are training set. 
Train your regression model on the training set and then use it to predict the outcome on the test set.
Calculate the RMSE (Root Mean Square Error) on the test set and return it.

In [ ]:
### Code

# 7. Data Assessment

## Section 3: Build a regression model



**Question:**

On the provided data, we only have the quantity of sold products. What other important data is missing to estimate the real demand? 


# Ask here

# 8. 

## Section 4: Model interpretation

We would like now to focus on a particular and rare product: 'Chia seeds'.

This product represent 2% of the sales volume (in terms of quantity),

Your colleague has build a classification model to predict if a given customer will buy this product.
The output of the algorithm is binary:
- 1 when the model predicts that the customer will buy 'Chia seeds'
- 0 when the model predicts that the customer will **NOT** buy 'Chia seeds'

The model your colleague made has a good accuracy:
- For Chia seeds buyers, the model is correct 98% of the time.
- For **NON** Chia seeds buyers, the model is correct 98% of the time.

You have run the model on a customer from your database, and the model predicted a positive answer meaning that he will buy chia seeds.


**Question:**

What is the probability of this customer to be a chia seeds buyer?

Pick **ONE** option:
- 99%
- 90%
- 80%
- 70%
- 50%
- 40%
- 30%
- 20%
- 10%
- 1%




# 9. 

## Section 5: Preprocessing Step

Imagine you have to develop a regression model.
After gathering all the data you need on one table, and after you build your features, you ended up with a table having 7000 observations and 8000 features.

What is your next step?
Can you provide 3 different techniques to do it?
Can you also explain the main differences between them?

Please provide your answer in the following editor.



# Ask here